# Transformar os dados RAW (Binance BTCUSDT, all trades) -> OHCL

In [1]:
raw_dataset_path = '../datasets/BTCUSDT-Trades/'
output_path = '../output'

In [2]:
import os
from loguru import logger
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import pandas as pd

logger.remove()
logger.add(lambda msg: print(msg, end=""), level="INFO")

from utils import create_dollar_bars

In [3]:
df_dask = dd.read_parquet(os.path.join(raw_dataset_path, 'BTCUSDT-Dataset-part-1.parquet'))

In [5]:
df_dask.head()

,trade_id,price,qty,quoteQty,time,isBuyerMaker,isBestMatch
0,0,4261.48,0.100000,426.148000,2017-08-17 04:00:28.322,True,True
1,1,4261.48,1.600000,6818.368000,2017-08-17 04:00:32.285,True,True
2,2,4261.48,0.075183,320.390851,2017-08-17 04:00:32.322,False,True
3,3,4280.56,0.029600,126.704576,2017-08-17 04:02:48.879,False,True
4,4,4280.56,0.231474,990.838345,2017-08-17 04:02:48.887,False,True


In [2]:
def dataSampler(barType="standard", samplerType="time", samplerAmount=100, maxRecords=None):
    barTypeDictionary = {
        "standard": "Padrão",
        "imbalance": "de Desequilíbrio",
        "run": "de Ordem Iceberg"
    }
    samplerDictionary = {
        "time": "Temporal",
        "ticks": "Ticks",
        "volume": "Volume",
        "dollar": "Dollar"
    }

    output_directory = os.path.join(output_path, f"{samplerType}-bars-[{samplerAmount}]")
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
        logger.info(f"Diretório criado: {output_directory}")

    print(f"Criando Barras {samplerDictionary[samplerType]} {barTypeDictionary[barType]} agrupadas a cada {samplerAmount}...")

    # Verificar se já existem arquivos .parquet no diretório de saída
    parquet_files_output = [f for f in os.listdir(output_directory) if f.endswith('.parquet')]
    if parquet_files_output:
        logger.info(f"'{output_directory}' já existe e contém arquivos .parquet. Carregando de {output_directory}...")

        # Carregar todos os arquivos .parquet usando Dask
        try:
            bars = dd.read_parquet(os.path.join(output_directory, '*.parquet')).compute()
            logger.info("'dollar_bars' carregado com sucesso.")
            return bars
        except Exception as e:
            logger.error(f"Erro ao carregar arquivos .parquet: {e}")

    logger.info("Criando 'dollar_bars'...")

    dollar_bars_path = os.path.join(output_directory, 'dollar_bars.parquet')

    # Obter a lista de todos os arquivos Parquet no raw_dataset_path
    parquet_files = [os.path.join(raw_dataset_path, f) for f in os.listdir(raw_dataset_path) if f.endswith('.parquet')]
    parquet_files.sort()

    if not parquet_files:
        logger.warning(f"Nenhum arquivo .parquet encontrado em '{raw_dataset_path}'.")
        return []

    logger.info(f"Total de arquivos .parquet a serem processados: {len(parquet_files)}")

    # Carregar todos os arquivos .parquet usando Dask diretamente
    try:
        df_dask = dd.read_parquet(os.path.join(raw_dataset_path, '*.parquet'))
        logger.info("Todos os arquivos .parquet foram carregados com sucesso.")
    except Exception as e:
        logger.error(f"Erro ao carregar arquivos .parquet: {e}")
        return []

    # Se maxRecords estiver definido, limitar o DataFrame
    if maxRecords is not None:
        df_dask = df_dask.head(maxRecords, compute=False)
        logger.info(f"Limite de registros definido para {maxRecords}.")

    # Criar e salvar 'dollar_bars'
    try:
        dollar_bars = create_dollar_bars(df_dask, samplerAmount, dollar_bars_path)
        logger.info("'dollar_bars' criado e salvo com sucesso.")
        return dollar_bars
    except Exception as e:
        logger.error(f"Erro ao criar 'dollar_bars': {e}")
        return []


# Realiza Amostragem

In [3]:
serieBars = dataSampler(
    barType="standard",
    samplerType="dollar",
    samplerAmount=10_000_000
)

sample_bars = serieBars.head()
display(sample_bars)

Criando Barras Dollar Padrão agrupadas a cada 10000000...
2024-12-09 11:29:26.535 | INFO     | __main__:dataSampler:35 - '../output/dollar-bars-[10000000]' já existe e contém arquivos .parquet. Carregando de ../output/dollar-bars-[10000000]...
2024-12-09 11:29:26.691 | INFO     | __main__:dataSampler:40 - 'dollar_bars' carregado com sucesso.


""


In [ ]:
df_dask = dd.read_parquet(os.path.join(raw_dataset_path, '*.parquet'))